<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph(2)_Persistence_and_Human_in_the_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph(2)_Persistence_HITL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangGraph (2): 状態の永続化とHuman-in-the-loop

前回の学習では、実行が終了すると状態（State）がリセットされるエージェントを作成しました。
今回は、実用的なアプリケーションに不可欠な以下の機能を実装します。

1.  **Persistence (永続化):** 会話履歴を保存し、プログラムを再起動しても会話を継続できるようにします。まずはメモリ上への保存、次にファイルへの保存を学びます。
2.  **Inspection (内部確認):** 保存されたデータベースの中身を直接確認します。
3.  **Human-in-the-loop (人間介入):** 処理を一時停止し、人間が承認や修正を行ってから再開するフローを構築します。

### 学習目標
* `MemorySaver` と `SqliteSaver` の違いと使い分け。
* SQLite データベースの直接操作によるデータ確認。
* `interrupt_before` と `update_state` によるプロセス制御。

### 1. 環境構築とAPI設定

SQLite 用のチェックポイントライブラリ `langgraph-checkpoint-sqlite` を追加でインストールします。

In [ ]:
!pip install -qU langgraph langgraph-checkpoint-sqlite langchain-openai langchain-community termcolor

import os
from google.colab import userdata
from langchain_openai import ChatOpenAI
from termcolor import cprint

try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    cprint("API Key loaded.", "green")
except Exception:
    cprint("Error: Please set OPENROUTER_API_KEY in secrets.", "red")

# LLMの初期化
llm = ChatOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    model="deepseek/deepseek-chat-v3-0324",
    temperature=0
)

## 2. Persistence 1: インメモリ保存 (MemorySaver)

LangGraph では `Checkpointer` という仕組みを用いて、グラフのステップごとの状態を保存します。
まずは最も基本的な、メモリ（RAM）上にデータを保存する `MemorySaver` を使用します。これは高速ですが、ランタイムが終了するとデータは消失します。

### グラフの定義

基本的なチャットボットのグラフ構造を作成します。

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END

# Stateの定義: 会話履歴をリストとして保持
class State(TypedDict):
    messages: Annotated[list, add_messages]

# ノードの定義: LLMの応答を生成
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

# グラフ構築
builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_edge(START, "chatbot")
builder.add_edge("chatbot", END)

### MemorySaver の適用

グラフのコンパイル時に `checkpointer` を引数として渡します。

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# メモリ上のCheckpointerを初期化
memory = MemorySaver()

# checkpointerを指定してコンパイル
graph_mem = builder.compile(checkpointer=memory)

### thread_id によるセッション管理

実行時の設定 `config` に `thread_id` を指定することで、会話のコンテキストを分離・維持できます。

In [ ]:
from langchain_core.messages import HumanMessage

# 設定: スレッドID "thread-1"
config_1 = {"configurable": {"thread_id": "thread-1"}}

print("--- Thread 1: Turn 1 ---")
# 初回の会話
input_msg = {"messages": [HumanMessage(content="私の名前はタナカです。")]}
for event in graph_mem.stream(input_msg, config=config_1):
    for value in event.values():
        print(f"Assistant: {value['messages'][-1].content}")

print("\n--- Thread 1: Turn 2 ---")
# 2回目の会話（前回の内容を覚えているか確認）
input_msg_2 = {"messages": [HumanMessage(content="私の名前を覚えていますか？")]}
for event in graph_mem.stream(input_msg_2, config=config_1):
    for value in event.values():
        print(f"Assistant: {value['messages'][-1].content}")

## 3. Persistence 2: SQLiteへの保存 (SqliteSaver)

実運用や、Colab の再起動後もデータを保持したい場合は、ファイルやデータベースに保存する必要があります。ここでは Python 標準で扱える `SqliteSaver` を使用します。

### SQLite データベースへの接続と保存

`sqlite3` でデータベースファイルを作成し、それを `SqliteSaver` に渡します。

In [ ]:
import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

# 1. DBファイルへの接続 (なければ作成されます)
# check_same_thread=False は Colab/Jupyter 環境でのエラー防止用
conn = sqlite3.connect("chat_history.db", check_same_thread=False)

# 2. Checkpointer の作成
sqlite_saver = SqliteSaver(conn)

# 3. コンパイル (保存機能を有効化)
# 先ほどと同じグラフ定義(builder)を使いますが、saverを切り替えます
graph_db = builder.compile(checkpointer=sqlite_saver)

print("Graph compiled with SqliteSaver.")

### 実行と保存

スレッドIDを指定して会話を行います。この内容は `chat_history.db` ファイルに書き込まれます。

Colab の左側メニュー「ファイル」アイコンから `chat_history.db` を右クリックしてダウンロードすることで、ローカル環境でも確認可能です。

In [ ]:
# 設定: スレッドID "user-db-1"
config_db = {"configurable": {"thread_id": "user-db-1"}}

print("--- DB Chat: Turn 1 ---")
inputs = {"messages": [HumanMessage(content="好きな色は青です。")]}

for event in graph_db.stream(inputs, config=config_db):
    for value in event.values():
        print(f"Assistant: {value['messages'][-1].content}")

## 4. SQLite ファイルの中身を見る

LangGraph を介さずに、作成された `chat_history.db` の中身を Python 標準の `sqlite3` ライブラリで直接確認します。LangGraph がどのように状態を保存しているか、内部構造を把握します。

In [ ]:
# 読み取り専用でカーソルを作成
cursor = conn.cursor()

# 1. テーブル一覧の取得
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print("Tables:", tables)

# 2. checkpoints テーブルの中身を確認 (最新1件)
# LangGraph は 'checkpoints' テーブルにシリアライズされた状態を保存します
print("\n--- Latest Checkpoint Data ---")
cursor.execute("SELECT thread_id, checkpoint_id, checkpoint FROM checkpoints ORDER BY checkpoint_id DESC LIMIT 1;")
row = cursor.fetchone()

if row:
    print(f"Thread ID: {row[0]}")
    print(f"Checkpoint ID: {row[1]}")
    # checkpoint カラムにはバイナリデータが入っています
    print(f"Raw Data (first 50 bytes): {row[2][:50]}...")
else:
    print("No data found.")

## 5. 保存データからの読み込み（再開）

保存されたデータベースがあれば、グラフオブジェクトを作り直しても会話を再開できます。
同じ `conn` (DB接続) と同じ `thread_id` を使用して確認します。

In [ ]:
print("--- DB Chat: Turn 2 (Restarted) ---")
inputs_2 = {"messages": [HumanMessage(content="私の好きな色がわかりますか？")]}

# 以前の会話履歴がロードされ、文脈を踏まえた回答が返ります
for event in graph_db.stream(inputs_2, config=config_db):
    for value in event.values():
        print(f"Assistant: {value['messages'][-1].content}")

## 6. 【演習】 永続化買い物リストボット

ここまでの知識を使って、簡単な演習を行います。

**課題:**
以下の要件を満たすボットを、穴埋めコードを完成させて作成してください。
1.  ユーザーが「みかんを追加」と言うと、リストに追加する。
2.  `shopping_db.sqlite` というファイルに状態を保存する。
3.  スレッドIDを変えると、別のリストになることを確認する。

In [ ]:
# 1. DB接続とSaverの作成
# ヒント: ファイル名は "shopping_db.sqlite" を指定
conn_shop = sqlite3.■■■("■■■", check_same_thread=False)
# ヒント: Sqlite で保存する
memory_shop = ■■■(conn_shop)

# 2. グラフのコンパイル
# ヒント: 作成した memory_shop を checkpointer に指定
shop_graph = builder.compile(■■■=■■■)

# 3. 実行確認 (User A)
config_a = {"configurable": {"thread_id": "user-a"}}

print("--- User A: Adding items ---")
shop_graph.invoke({"messages": [HumanMessage(content="買い物リストに「牛乳」を追加して")]}, config=■■■)
response = shop_graph.invoke({"messages": [HumanMessage(content="リストの中身を教えて")]}, config=■■■)
print(f"User A List: {response['messages'][-1].content}")

# 4. 実行確認 (User B)
# ヒント: User A とは異なる thread_id を設定
config_b = {"configurable": {"thread_id": "■■■"}}

print("\n--- User B: New List ---")
response_b = shop_graph.invoke({"messages": [HumanMessage(content="リストの中身を教えて")]}, config=■■■)
print(f"User B List: {response_b['messages'][-1].content}")

# 解説:
# User B のリストには「牛乳」が入っていない（User Aとは独立している）ことが確認できれば成功です。

## 7. Human-in-the-loop (1): 承認とフィードバックループ

ここからは、エージェントの処理に人間が介入する **Human-in-the-loop (HITL)** 機能の実装に入ります。
LangGraph では `interrupt_before` を使うことで、指定したノードの実行直前で処理を一時停止できます。

実運用では、単に処理を止めて承認する（YES/NO）だけでなく、**「もっとこうしてほしい」という修正指示（フィードバック）を与え、内容を改善してから処理を再開する** ケースが重要です。

### シナリオ: メール送信前の査読と修正

1.  **draft:** ユーザーの指示でメール本文を作成する。
2.  **(一時停止):** 人間が内容を確認し、必要なら修正指示を出す。
3.  **send:** メールを送信する（表示のみ）。

ここでは、フィードバックがあった場合に **「グラフの外で修正案を作成し、状態を差し替える」** 手法を解説します。

In [ ]:
# ノード定義
def draft_node(state: State):
    last_msg = state["messages"][-1].content
    # 初稿を作成
    response = llm.invoke(f"以下の要望で短いメール下書きを作成して:\n{last_msg}")
    return {"messages": [response]}

def send_node(state: State):
    # 直前のメッセージ（作成されたメール本文）を取得
    email_content = state["messages"][-1].content
    print(f"\n📧 TRANSMITTING:\n{email_content}\n")
    return {"messages": [HumanMessage(content="送信完了しました。")]}

# グラフ構築（シンプルな直線的なフロー）
email_builder = StateGraph(State)
email_builder.add_node("draft", draft_node)
email_builder.add_node("send", send_node)

email_builder.add_edge(START, "draft")
email_builder.add_edge("draft", "send")
email_builder.add_edge("send", END)

# コンパイル (中断設定)
# interrupt_before=["send"] により、sendノードに入る直前（draftの出力後）で停止します
email_graph = email_builder.compile(
    checkpointer=memory,
    interrupt_before=["send"]
)

# 可視化
from IPython.display import Image, display
display(Image(email_graph.get_graph().draw_mermaid_png()))

### ステップ1: 初稿の生成と中断

以下のセルを実行すると、下書きが作成された時点で処理が停止します。

In [ ]:
config_email = {"configurable": {"thread_id": "email-feedback-1"}}

print("--- Step 1: Draft Creation ---")

# 実行開始
# send ノードの前で停止するため、"TRANSMITTING" は表示されません
for event in email_graph.stream(
    {"messages": [HumanMessage(content="プロジェクトの進捗報告。順調だが、デザインのみ遅れている。")]},
    config=config_email
):
    for node, values in event.items():
        if isinstance(values, dict) and "messages" in values:
            print(f"Draft:\n{values['messages'][-1].content}")

print("\n--- 🛑 Stopped at Breakpoint ---")

## 8. Human-in-the-loop (2): AIによる修正と状態更新

下書きの内容を確認します。もし修正が必要な場合、人間が全て書き直すのではなく、**「もっと丁寧にして」といった指示を出し、AIに書き直させる** のが効率的です。

`update_state` メソッドを使用すると、グラフの実行履歴（State）を書き換えることができます。ここでは、外部で修正版を作成し、それを「draftノードが生成した結果」として現在のStateに上書きすることで、修正を反映させます。

以下のコードは、**現在のState取得 → 指示に基づく修正生成 → State更新** というフローを実行します。

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

print("--- Step 2: Feedback & Refinement ---")

# 1. 現在のState（直前に生成された下書き）を取得
snapshot = email_graph.get_state(config_email)
current_draft = snapshot.values["messages"][-1].content

# 2. 人間による修正指示 (Feedback)
feedback = "顧客向けなので、もっと丁寧な敬語に書き換えて。" # @param {"type":"string"}
print(f"Feedback: {feedback}")

# 3. 指示に基づいて修正版を作成 (グラフ外でのLLM実行)
# 修正用のプロンプトを定義
rewrite_prompt = ChatPromptTemplate.from_template(
    """
    以下のメール下書きを、指示に従って書き直してください。

    [元のメール]
    {draft}

    [修正指示]
    {instruction}
    """
)
# 修正チェーンの実行
rewrite_chain = rewrite_prompt | llm
new_draft_msg = rewrite_chain.invoke({
    "draft": current_draft,
    "instruction": feedback
})

print(f"\n✨ Revised Draft:\n{new_draft_msg.content}")

# 4. Stateを更新
# draftノードの出力を、この新しいメッセージに置き換えます
# これにより、グラフは「最初からこの完璧なメールが生成された」かのように振る舞います
email_graph.update_state(
    config_email,
    {"messages": [new_draft_msg]}, # 修正後のメッセージリスト
    as_node="draft"                # draftノードの出力として扱う
)

print("\n✅ State Updated. Ready to resume.")

### ステップ3: 処理の再開

修正された状態から処理を再開します。`stream(None, ...)` を実行すると、停止していた `send` ノードから処理が始まります。
データは修正済みのものになっているため、送信される内容は「丁寧な敬語版」になります。

In [ ]:
print("--- Step 3: Approve & Send ---")

# 停止中のグラフを再開
# Stateは既に更新されているため、修正後の内容で send ノードが実行されます
for event in email_graph.stream(None, config=config_email):
    for node, values in event.items():
        if isinstance(values, dict) and "messages" in values:
            print(f"System: {values['messages'][-1].content}")

## 9. 【演習】 旅行プランの修正と予約

同様の仕組みを用いて、旅行予約エージェントを作成します。

**シナリオ:**
1.  **planner:** ユーザーの要望を受けて旅行プランを提案する。
2.  **(一時停止):** 人間が確認し、予算や条件の変更指示を出す。
3.  **booker:** 予約完了メッセージを出す。

**課題:**
以下のコードの `■■■` を埋めて、**「プラン提案」→「人間による条件変更（もっと安くして等）」→「AIによる再プランニングとState更新」→「予約実行」** という流れを完成させてください。

In [ ]:
# 1. ノード定義
def planner_node(state: State):
    # ユーザーの要望に基づきプランを提案
    return {"messages": [llm.invoke(state["messages"])]}

def booker_node(state: State):
    # 最新のメッセージ（確定したプラン）を取得
    if state["messages"]:
        plan = state["messages"][-1].content
    else:
        plan = "プランなし"
    print(f"\n✈️ BOOKING CONFIRMED for:\n{plan}\n")
    return {"messages": [HumanMessage(content="予約手続きが完了しました。")]}

# 2. グラフ構築
travel_builder = StateGraph(State)
travel_builder.add_node("planner", planner_node)
travel_builder.add_node("booker", booker_node)
travel_builder.add_edge(START, "planner")
travel_builder.add_edge("planner", "booker")
travel_builder.add_edge("booker", END)

# 3. コンパイル (bookerの前で停止)
memory = MemorySaver()
travel_graph = travel_builder.compile(
    checkpointer=memory,
    interrupt_before=["■■■"] # ここで停止させるノードを指定
)

# --- 実行フロー ---
config_trip = {"configurable": {"thread_id": "trip-fix-1"}}

print("--- 1. Initial Planning ---")
# 初回のプラン作成
for event in travel_graph.stream(
    {"messages": [HumanMessage(content="京都の1泊2日旅行プランを提案して。高級旅館に泊まりたい。")]},
    config=config_trip
):
    for node, values in event.items():
        if isinstance(values, dict) and "messages" in values:
            print(f"Planner Output:\n{values['messages'][-1].content}")

print("\n--- 🛑 Stopped. Feedback Required. ---")

# --- ここから演習箇所 ---

# 4. 修正プロセスの実装
# A. 現在のStateから、直前のプラン内容を取得
snapshot = travel_graph.get_state(config_trip)
current_plan = snapshot.values["messages"][■■■].content
print(current_plan)

In [ ]:
# B. 修正指示
feedback = "予算オーバーです。1万円以下のビジネスホテルに変更して、食事はB級グルメ中心にして。" # @param {"type":"string"}

In [ ]:
# C. AIに書き直しをさせる (Prompt と Chain の作成)
fix_prompt = ChatPromptTemplate.from_template(
    """
    以下の旅行プランに対し、ユーザーからの修正指示があります。
    指示を反映した新しいプランを全文書き直してください。

    [元のプラン]
    {plan}

    [修正指示]
    {instruction}
    """
)
# チェーンを作成して実行 (invoke)
fix_chain = fix_prompt | llm
new_plan_msg = fix_chain.■■■({
    "plan": current_plan,
    "instruction": feedback
})

print(f"\n✨ Revised Plan:\n{new_plan_msg.content}")

# D. Stateを更新
# plannerノードが修正案を出したことにして、履歴を書き換える
travel_graph.update_state(
    config_trip,
    {"messages": [■■■]}, # 新しいメッセージを指定
    as_node="■■■"        # どのノードの出力として扱うか
)
print("✅ State Updated.")

# 5. 処理の再開 (予約実行)
print("\n--- 2. Booking ---")
for event in travel_graph.stream(None, config=config_trip):
    for node, values in event.items():
         if isinstance(values, dict) and "messages" in values:
            print(f"Planner Output:\n{values['messages'][-1].content}")

### まとめ

本セクションでは、Human-in-the-loop の応用として、単なる承認だけでなく **途中結果の修正** を実装しました。

1.  `interrupt_before` で重要なアクション（送信、予約など）の前に停止する。
2.  `get_state` で現在の状況を確認する。
3.  LLMを用いて、人間のフィードバックを反映した修正データを作成する。
4.  `update_state` でグラフ内の状態を更新し、処理を再開する。

これにより、AIの自律性と人間の監督・修正を組み合わせた、安全かつ高品質なアプリケーションが構築可能になります。